In [15]:
""" 
    homework3 for Machine Learning 
    author: Robert Simari <rsimari>
    date: 2/13/18
    purpose: builds a decision tree based on heart data
"""

from math import log
import graphviz as gv

def compute_entropy(data):
    """
        @param [list[list]]: 2D array of data
        @return [float]: entropy of given data
        Note: assumes that the outcome is the final column in each list
        entropy(Y) = -p_1 * log_2(p_1) - p_2 * log_2(p_2) - ...
    """
    total_instances = len(data)
    outcome_index = len(data[0]) - 1
    
    # all unique outcomes possible, assumes nominal
    outcomes = list(set([arr[outcome_index] for arr in data]))
    
    # gives counts for all the outcomes possible for a particular set of instances with a feature value
    outcome_count = {}
    for arr in data:
        if arr[outcome_index] in outcome_count:
            outcome_count[arr[outcome_index]] += 1
        else:
            outcome_count[arr[outcome_index]] = 1
            
    # computes entropy sum using each outcome possible (assuming nominal outcomes)
    entropy = 0
    for _, count in outcome_count.items():
        r = count * 1.0 / total_instances
        entropy = entropy - r * log(r, 2)
        
    return entropy


def compute_cond_entropy(feature_name, data, split_func = None):
    """
        @param [string, string, list[list], function]: name of feature (see feature_names list), 2D list of values, an optional
        function to split a continuous feature into bins
        @return [float]: returns conditional entropy value of Y given X
        H(Y|X) = sum(P(x_k = v_j) * H(Y|x_k = v_j)) 
                            or
        entropy(Y | x_k) = for each x_k: sum((# of instances of x_k)/(# of instances) * entropy(x_k, x_k instances))
    """
    # to compute entropy(outcome, x_k instances):
    #     1. select all cases with x_k, call it data2
    #     2. return compute_entropy(x_k, data2)
    if feature_name in feature_names:
        index = feature_names.index(feature_name)
    else:
        return -1 # feature is unknown
    
    total_instances = len(data)
    cond_entropy = 0

    # getting all possible values/bins
    if split_func == None:
        bins = list(set([arr[index] for arr in data]))
    else:
        bins = list(set(split_func(arr[index]) for arr in data)) # split function accounts for continuous features
    
    # for every feature value/bin
    for value in bins:
        # compute P(x_k = v_j)
        if split_func == None:
            feature_instances = [arr for arr in data if arr[index] == value] # nominal, instances that have our desired value
        else:
            feature_instances = [arr for arr in data if split_func(arr[index]) == value] # continuous, count instances of x_k
        count = len(feature_instances)
        prob = count * 1.0 / total_instances
        
        # compute H(Y|x_k = v_j)
        entropy = compute_entropy(feature_instances)
        
        # add to sum for conditional entropy
        cond_entropy += (entropy * prob)
    
    return cond_entropy

def compute_inf_gain(feature_name, data, split_func = None):
    """
        @param [string, list[list], function]: name of column for feature, 2D list of data, function for splitting
        continuous features
        @return [float]: information gain of Y given X
        infGain(Y | x_k) = entropy(Y) - entropy(Y | x_k)
    """
    return compute_entropy(data) - compute_cond_entropy(feature_name, data, split_func)

def compute_max_inf_gain(data, features):
    """
        @param [list[list], list]: 2D list of data, the list of features to compare
        @return [string, float]: name of feature, information gain of feature
    """
    mx = -1
    for feature in features:
        # check if split function is needed?
        ig = compute_inf_gain(feature, data)
        mx = max(mx, ig)
        if ig == mx: 
            mx_feature = feature
    return mx_feature, mx


In [16]:
# split functions for each feature

def split_nominal(value, feature): # organizes the nominal features into discrete bins
    feature_index = feature_names.index(feature)
    all_vals = list(set([arr[feature_index] for arr in data]))
    return all_vals.index(value)

def split_age(age):
    if float(age) > 50:
        return 1 # returns a bin number for an age to be place in
    return 0

def split_sex(sex):
    return split_nominal(sex, "sex")

def split_chest(chest):
    return split_nominal(chest, "chest")

def split_rbp(rbp):
    if float(rbp) > 130:
        return 1
    return 0

def split_sc(sc):
    if float(sc) > 250:
        return 1
    return 0

def split_fbs(fbs):
    return split_nominal(fbs, "fbs")

def split_rer(rer):
    return split_nominal(rer, "rer")

def split_maxhr(maxhr):
    if float(maxhr) > 120:
        return 1
    return 0

def split_eia(eia):
    return split_nominal(eia, "eia")

def split_oldpeak(oldpeak):
    if float(oldpeak) > 1:
        return 1
    return 0

def split_slopest(slopest):
    return split_nominal(slopest, "slopest")

def split_vessels(vessels):
    return split_nominal(vessels, "vessels")

def split_thal(thal):
    return split_nominal(thal, "thal")

# dict maps each feature to its split function
split_dict = {
    "age" : split_age,
    "sex" : split_sex, 
    "chest": split_chest, 
    "rbp" : split_rbp, 
    "sc" : split_sc, 
    "fbs" : split_fbs, 
    "rer" : split_rer, 
    "maxhr" : split_maxhr, 
    "eia" : split_eia, 
    "oldpeak" : split_oldpeak, 
    "slopest" : split_slopest, 
    "vessels" : split_vessels,
    "thal" : split_thal  
}

In [24]:
# Build Decision Tree Using Information Gain

class Decision_Node(object): # possibly a wrapper for the graphix pkg node
    def __init__(self):
        self.children = []
        self.split_feature = ""
        
    def add_child(self, child):
        self.children.append(child)
        
    def is_leaf(self):
        return len(self.children) == 0
        
class Decision_Tree(object):
    def __init__(self, data, feature_names):
        self.root = Decision_Node()
        self.features = feature_names
        self.data = data
    
    def build_tree(self):
        if len(self.data) == 0:
            return
        self.build_next_split(self.root, self.features, self.data)
    
    def build_next_split(self, root, features, data):
        if len(features) == 0 or len(data) == 0:
            return

        feature, inf_gain = compute_max_inf_gain(data, features)
        # get index of feature in data 
        feature_index = feature_names.index(feature)
        # get index of feature in local feature list (features)
        feature_index2 = features.index(feature)
        # set split by feature for the current node
        root.split_feature = feature
        # create children, based on split functions
        split_func = split_dict[feature]
        bins = list(set([split_func(arr[feature_index]) for arr in data]))
        for b in bins:
            child = Decision_Node()
            root.add_child(child)
            # filter data that only has the desired feature values/bin of the branch
            data2 = [arr for arr in data if split_func(arr[feature_index]) == b]
            # recurse on each branch with the filtered feature list 
            self.build_next_split(child, features[:feature_index2] + features[feature_index2+1:], data2)

        # remove feature from being in future splits (assumes that continuous features only split once)
        features.remove(feature)
        return
    
    def make_prediction(self, data):
        pass
    
    def view(self, file = 'dt.gv'):
        graph = gv.Digraph('DecisionTree', filename = file)
        q = []
        q.insert(0, self.root)
        while len(q) != 0:
            node = q.pop()
            for child in node.children:
                graph.edge(node.split_feature, child.split_feature)
                q.insert(0, child)
        graph.view()
        
        

In [27]:
feature_names = ["age", "sex", "chest", "rbp", "sc", "fbs", "rer", "maxhr", "eia", "oldpeak", "slopest", "vessels", "thal"]

TEST_MODE = False

data = []

with open("heart.data") as f:
    line = f.readline().rstrip()
    while line:
        cols = line.split()
        data.append(cols)
        line = f.readline().rstrip()

if TEST_MODE:
    feature_names = ['school']
    test = [['ND', 'YES'], ['MSU', 'NO'], ['ND', 'NO'], ['ND', 'YES'], ['ND', 'NO'], ['USC', 'YES'], ['MSU', 'NO'], ['USC', 'YES']]
    assert(compute_entropy(test) == 1)
    assert(compute_cond_entropy('school', test) == 0.5)
    assert(compute_max_inf_gain(test) == ('school', 0.5))
    t
tree = Decision_Tree(data, feature_names)
tree.build_tree()
# assert(tree.root.split_feature == 'sc')
# tree.view()